In [2]:
import pandas as pd 

fake_df = pd.read_csv('data/Fake.csv')
real_df = pd.read_csv('data/True.csv')

In [3]:
fake_df['label'] = 0
real_df['label'] = 1

In [4]:
df = pd.concat([fake_df, real_df]).reset_index(drop=True)

In [ ]:
print(df.isnull().sum())
print(df.duplicated().sum())

In [ ]:
print(df['label'].value_counts())

In [ ]:
import matlpotlib.pyplot as plt 

df['label'].value_counts().plot(kind='bar', title='Real vs Fake News')
plt.xticks(ticks=[0,1], labels=['Fake', 'Real'], rotation=0)
plt.show()